In [1]:
import pandas as pd
import numpy as np

In [35]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.cluster import KMeans

In [2]:
cc_concat = pd.read_csv('data/cleaned_cc_tweets.csv')

In [4]:
cc1 = pd.read_csv('data/cc_tweets1.csv')
cc2 = pd.read_csv('data/cc_tweets2.csv')

C:\Users\samtg\AppData\Local\Temp\ipykernel_37184\2144096723.py:1: DtypeWarning: Columns (28,29,31,39,41,42,43,49,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  cc1 = pd.read_csv('data/cc_tweets1.csv')
C:\Users\samtg\AppData\Local\Temp\ipykernel_37184\2144096723.py:2: DtypeWarning: Columns (28,29,31,39,41,42,43,49,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  cc2 = pd.read_csv('data/cc_tweets2.csv')


In [8]:
current_cc = [
    'SandraForNY1', 'LincolnRestler', 'BobHoldenNYC', 'OswaldFeliz', 'AdrienneCD28',
    'KalmanYegerNYC', 'DianaAyalaNYC', 'mvelaznyc', 'KevinCRiley', 'CMMercedesCD46',
    'CMCarlinaRivera', 'Lynn4NYC', 'juliej_won', 'PiSanchezNYC', 'GaleBrewerNYC',
    'crystalrhudson', 'voteshekar', 'ChrisMarteNYC', 'NYCCouncil38', 'DarleneMealy55',
    'Salamancajr80', 'AriKagan47', 'OsseChi', 'KamillahMHanks', 'CarlinaRivera',
    'AmandaFariasNYC', 'KeithPowersNYC', 'CMDarleneMealy', 'ShahanaFromBK',
    'InnaVernikov', 'Powers4Queens', 'KalmanYeger', 'FranciscoMoyaNY', 'JimGennaro',
    'alexaforcouncil', 'CMNantashaW', 'JoeBorelliNYC', 'CnDelarosa', 'JustinBrannan',
    'EricDinowitz', 'ShaunAbreu', 'DarleneMealy', 'JulieMenin', 'ebottcher',
    'DMCarr', 'JoannAriola32', 'galeabrewer', 'FarahNLouis',
    'CMShekarK', 'CMRitaJoseph', 'CouncilwomanKrj', 'CMvpaladino', 'CMPiSanchez',
    'CMJenGutierrez', 'cmmvelaz', 'CMbarron2', 'CMSandyNurse', 'CMBrooksPowers',
    'A_StevensD16', 'CMShaunAbreu', 'CMLindaLee', 'CMSandraUng', 'CMCrystalHudson',
    'CMRestler', 'CMChiOsse', 'CMAriKagan47', 'CMJulieWon', 'CabanD22', 'CMDMCarr',
    'CMSalamancaJr', 'CMKevinCRiley', 'CMAmandaFarias', 'EricDinowitzNYC',
    'CMFranciscoMoya', 'NYCCouncilAyala', 'AdrienneToYou', 'NYCSpeakerAdams',
    'CMShahanaHanif', 'VickieforNYC', 'LindaLeeforNYC', 'JenGutierrezNYC',
    'CarmenForNYC', 'SandyforCouncil', 'RitaJosephNYC', 'CMFarahLouis', 'CMBobHoldenNYC',
    'althea4theBX', 'nycpa', 'tiffany_caban'
]


In [10]:
cc1_only = cc1[cc1['author.username'].isin(current_cc)]
cc2_only = cc2[cc2['author.username'].isin(current_cc)]

In [15]:
#check size of each df
print(cc2.shape)

(880968, 83)


In [14]:
#concat the two dfs
cc_concat = pd.concat([cc1_only, cc2_only])

In [25]:
cc_concat.rename(columns={'created_at': 'date',
                          'public_metrics.retweet_count': 'retweets', 
                          'in_reply_to_username': 'reply_to',
                          'retweeted_username': 'retweeted',
                          'author.username': 'username', 
                          'author.name': 'name',
                          'author.verified': 'verified', 
                          'public_metrics.like_count': 'likes', 
                          'public_metrics.quote_count': 'quotes', 
                          'public_metrics.reply_count': 'replies',
                           'author.description': 'user_bio'},
                            inplace=True)

In [26]:
cc_concat = cc_concat[['date', 'username', 'name', 'text', 'retweets',
           'likes', 'replies',  'quotes', 'user_bio','reply_to', 'retweeted']]

In [36]:
def get_sentiment_score(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity


In [79]:
police = ['cop', 'police', 'nypd','officer']
tax = ['tax','taxes']
bail = ['bail reform']
jail = ['jail','rikers']
bus = ['bus','bus lane']
bike = ['bike','bike lane','protected bike lane','cycle','cyclist']
housing = ['rent control','good cause eviction''affordable housing']
homeless = ['homeless','homelessness','homeless shelter']
rezone = ['rezoning','rezoned','upzoning','upzoned']
unions = ['dc37','1199seiou']

In [68]:
catagories = [police, bail, jail, bus, bike, housing, homeless, rezone, unions]

In [38]:
cop_tweets = cc_concat[cc_concat['text'].str.contains('|'.join(police), case=False)]

In [40]:
cop_tweets['sentiment_score'] = cop_tweets['text'].apply(get_sentiment_score)

C:\Users\samtg\AppData\Local\Temp\ipykernel_37184\425138969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cop_tweets['sentiment_score'] = cop_tweets['text'].apply(get_sentiment_score)


In [54]:
cop_username = cop_tweets.groupby('username')['sentiment_score'].mean().sort_values(ascending=False).reset_index()

In [61]:
sentimentAnalyser = SentimentIntensityAnalyzer()

In [62]:
def calculate_sentiment(text):
    scores = sentimentAnalyser.polarity_scores(text)
    compound_score = scores['compound']
    return compound_score

In [63]:
cop_tweets['vader_sentiment_score'] = cop_tweets['text'].apply(calculate_sentiment)

C:\Users\samtg\AppData\Local\Temp\ipykernel_37184\991218961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cop_tweets['vader_sentiment_score'] = cop_tweets['text'].apply(calculate_sentiment)


In [80]:
categories = [police, tax, bail, jail, bus, bike, housing, homeless, rezone, unions]
category_names = ['police','tax', 'bail', 'jail', 'bus', 'bike', 'housing', 'homeless', 'rezone', 'unions']

In [93]:
def calculate_weighted_sentiment_scores(df, categories, category_names):

    scores_df = pd.DataFrame()


    for category, name in zip(categories, category_names):
        subset = df[df['text'].str.contains('|'.join(category), case=False, na=False)]
        subset[f'{name}_score'] = subset['text'].apply(calculate_sentiment)

        # Perform groupby and calculate weighted average
        weighted_avg = subset.groupby('username').apply(lambda x: np.average(x[f'{name}_score'], weights=x['text'].str.len()))
        
        # Convert result to DataFrame
        weighted_avg = pd.DataFrame(weighted_avg, columns=[f'{name}_score']).reset_index()

        if scores_df.empty:
            scores_df = weighted_avg
        else:
            scores_df = pd.merge(scores_df, weighted_avg, on='username', how='left')

    scores_df.fillna(0, inplace=True)
    return scores_df


In [94]:
result_df = calculate_weighted_sentiment_scores(cc_concat, categories, category_names)

C:\Users\samtg\AppData\Local\Temp\ipykernel_37184\1537008052.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset[f'{name}_score'] = subset['text'].apply(calculate_sentiment)
C:\Users\samtg\AppData\Local\Temp\ipykernel_37184\1537008052.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset[f'{name}_score'] = subset['text'].apply(calculate_sentiment)
C:\Users\samtg\AppData\Local\Temp\ipykernel_37184\1537008052.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [95]:
X = result_df.drop('username', axis=1)
k = 5
kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
result_df['cluster'] = kmeans.labels_

c:\Users\samtg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [96]:
result_df

,username,police_score,tax_score,bail_score,jail_score,bus_score,bike_score,housing_score,homeless_score,rezone_score,unions_score,cluster
0,A_StevensD16,0.432458,0.810400,0.000000,0.000000,0.752289,0.691631,0,0.000000,0.0000,0.000000,4
1,AdrienneToYou,0.968600,0.000000,0.000000,0.000000,0.202455,0.796400,0,-0.025800,0.0000,0.000000,3
2,AmandaFariasNYC,-0.224849,0.583700,0.000000,0.000000,0.631566,0.584603,0,0.000000,0.0000,0.000000,4
3,AriKagan47,0.443980,0.450559,0.745000,-0.656687,0.503887,0.135596,0,0.611156,0.0000,0.000000,2
4,BobHoldenNYC,0.214182,0.155535,-0.711779,-0.234929,0.016653,0.125999,0,0.221807,0.0000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
79,juliej_won,0.006597,0.309338,0.000000,-0.461400,0.342982,0.089021,0,-0.050392,0.7500,0.419900,1
80,mvelaznyc,0.129516,0.443820,0.000000,0.000000,0.340498,0.129281,0,0.000000,0.0000,0.000000,4
81,nycpa,0.064548,0.688290,0.203059,-0.150835,0.155686,0.702832,0,0.234642,0.0000,0.000000,4
82,tiffany_caban,0.021752,-0.086468,0.000000,-0.097301,0.141296,-0.877300,0,-0.092957,0.0000,0.000000,0
